In [3]:
source("./aux.R")

Loading required package: NLP



Loading required package: koRpus.lang.en

Loading required package: koRpus

Loading required package: sylly

For information on available language packages for 'koRpus', run

  available.koRpus.lang()

and see ?install.koRpus.lang()



Attaching package: ‘koRpus’


The following object is masked from ‘package:tm’:

    readTagged



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [4]:
dataset <- read.csv("six_label_dataset.csv", col.names = c("Label", "Text", "Tag"))
dataset$Label <- change_labels(dataset$Label)
classes <- as.integer(sort(unique(dataset$Label)))
args <- sort(unique(unlist(strsplit(dataset$Tag, ","))))

In [5]:
occ_threshold <- 1
vocabulary <- get_vocabulary(dataset$Text, occ_threshold)
length(vocabulary)

[1] 21678

In [6]:
dataset$Text <- clean(dataset$Text)
dataset <- clean_empty_rows(dataset)

In [7]:
occ_threshold <- 1
vocabulary <- get_vocabulary(dataset$Text, occ_threshold)
length(vocabulary)

[1] 5142

In [44]:
seventy_percent <- floor(length(dataset$Text) * 0.7)
eightyfive_percent <- floor(length(dataset$Text) * 0.85)
n <- nrow(dataset)

training_set <- dataset[1:seventy_percent, ]
validation_set <- dataset[(seventy_percent + 1):eightyfive_percent, ]
testing_set <- dataset[(eightyfive_percent + 1):n, ]

In [45]:
occ_threshold <- 1
model <- train_multinomial_nb(classes, training_set, occ_threshold)

In [46]:
docs <- validation_set$Text

pred_labels <- sapply(docs, function(doc) {
  apply_multinomial_nb(classes, model$vocab, model$prior, model$condprob, doc)
})

In [47]:
correct_predictions <- sum(validation_set$Label == pred_labels)
total_predictions <- length(validation_set$Label)
accuracy <- correct_predictions / total_predictions

cat("Accuracy:", accuracy)

Accuracy: 0.218099

In [8]:
eigthy_percent <- floor(length(dataset$Text) * 0.8)
n <- nrow(dataset)

dataset <- dataset[sample(n), ]

training_set <- dataset[1:eigthy_percent, ]
testing_set <- dataset[(eigthy_percent + 1):n, ]

In [9]:
crossval_results <- kfold_cross_validation(training_set, k = 5, occ_thresholds = 1:20)
print(crossval_results)

   occ_threshold mean_accuracy
1              1     0.2244349
2              2     0.2215027
3              3     0.2219914
4              4     0.2201588
5              5     0.2186927
6              6     0.2172266
7              7     0.2157605
8              8     0.2162492
9              9     0.2162492
10            10     0.2168601
11            11     0.2169823
12            12     0.2162492
13            13     0.2172266
14            14     0.2166158
15            15     0.2173488
16            16     0.2175932
17            17     0.2173488
18            18     0.2177153
19            19     0.2195480
20            20     0.2191814


In [9]:
best_threshold <- crossval_results$occ_threshold[which.max(crossval_results$mean_accuracy)]
model <- train_multinomial_nb(classes, training_set, best_threshold)

In [10]:
pred_labels <- sapply(testing_set$Text, function(doc) {
  apply_multinomial_nb(classes, model$vocab, model$prior, model$condprob, doc)
})

In [11]:
correct_predictions <- sum(testing_set$Label == pred_labels)
total_predictions <- length(testing_set$Label)
accuracy <- correct_predictions / total_predictions

cat("Accuracy:", accuracy)

Accuracy: 0.2412109

In [12]:
confusion_matrix <- table(True = testing_set$Label, Predicted = pred_labels)
cat("Confusion Matrix:\n")
print(confusion_matrix)

Confusion Matrix:
    Predicted
True   0   1   2   3   4   5
   0  22  30  32  39  20  18
   1  27 104  66 104  67  49
   2  24  67  67  76  41  34
   3  18  74  56 128  85  43
   4   9  65  50  95 107  61
   5  18  61  49  86  90  66


In [20]:
crossval_results <- kfold_cross_validation_tags(training_set, k = 5, occ_thresholds = 1:20)
print(crossval_results)

   occ_threshold mean_accuracy
1              1     0.2151497
2              2     0.2131949
3              3     0.2142944
4              4     0.2134392
5              5     0.2130727
6              6     0.2180819
7              7     0.2186927
8              8     0.2177153
9              9     0.2160049
10            10     0.2194258
11            11     0.2185706
12            12     0.2184484


In [21]:
best_threshold <- crossval_results$occ_threshold[which.max(crossval_results$mean_accuracy)]
model <- train_multinomial_nb(classes, training_set, best_threshold)

In [22]:
pred_labels <- sapply(testing_set$Text, function(doc) {
  apply_multinomial_nb(classes, model$vocab, model$prior, model$condprob, doc)
})

In [23]:
correct_predictions <- sum(testing_set$Label == pred_labels)
total_predictions <- length(testing_set$Label)
accuracy <- correct_predictions / total_predictions

cat("Accuracy:", accuracy)

Accuracy: 0.2265625

In [24]:
confusion_matrix <- table(True = testing_set$Label, Predicted = pred_labels)
cat("Confusion Matrix:\n")
print(confusion_matrix)

Confusion Matrix:
    Predicted
True   0   1   2   3   4   5
   0  18  37  26  37  23  20
   1  26 118  54 100  72  47
   2  35  66  46  84  43  35
   3  22  69  52 115  83  63
   4  15  56  52  88 105  71
   5  30  60  40  90  88  62
